In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import os

from keras.optimizers.schedules import PolynomialDecay

from tf_agents.environments import suite_gym
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.policies.policy_saver import PolicySaver
from tf_agents.utils.common import function, element_wise_squared_loss

In [2]:
env_name = "LunarLander-v2"

#Agent parameters
learning_rate = 1e-3
update_period = 1200
discount_factor = 1
step_counter = tf.Variable(0)

#Buffer params
replay_buffer_capacity = 10000

#Training params
num_iterations = 150000

#Dataset params
batch_size = 64

# Environment

In [3]:
train_env_py = suite_gym.load(env_name)
eval_env_py = suite_gym.load(env_name)

In [4]:
train_env = TFPyEnvironment(train_env_py)
eval_env = TFPyEnvironment(eval_env_py)

In [5]:
train_env.observation_spec()

BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))

In [6]:
train_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(3, dtype=int64))

# Network

In [5]:
fc_layer_params = (64, 32)

q_net = QNetwork(train_env.observation_spec(),
                 train_env.action_spec(),
                 fc_layer_params = fc_layer_params,
                 activation_fn = tf.keras.activations.relu,
                 kernel_initializer = "he_normal" )

# Agent

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

epsilon_fn = PolynomialDecay(initial_learning_rate = 1.0,
                             decay_steps = 6,
                             end_learning_rate = 0.01)

agent = DqnAgent(train_env.time_step_spec(),
                 train_env.action_spec(),
                 q_network = q_net,
                 optimizer = optimizer,
                 target_update_period = update_period,
                 td_errors_loss_fn = element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                 gamma = discount_factor,
                 train_step_counter = step_counter,
                 epsilon_greedy = lambda : epsilon_fn(step_counter)) 

agent.initialize()

# Replay Buffer

In [7]:
replay_buffer = TFUniformReplayBuffer(data_spec = agent.collect_data_spec,
                                      batch_size= train_env.batch_size,
                                      max_length= replay_buffer_capacity)

In [8]:
observer = replay_buffer.add_batch

# Metrics

In [9]:
training_metrics = [tf_metrics.AverageReturnMetric(),
                 tf_metrics.NumberOfEpisodes()]
#Might need a logger to print them to screen (cf. logging in Geron book)

# Drivers

In [10]:
random_policy = RandomTFPolicy(time_step_spec = train_env.time_step_spec(),
                               action_spec = train_env.action_spec())

In [13]:
initial_collect_driver = DynamicStepDriver(train_env,
                                           random_policy,
                                           observers = [observer] + training_metrics,
                                           num_steps = replay_buffer_capacity)

In [15]:
good_policy = tf.saved_model.load("Good policies/policy_120k_huber")

pre_trained_driver = DynamicStepDriver(train_env,
                                       good_policy, 
                                       observers = [observer] + training_metrics,
                                       num_steps = replay_buffer_capacity)

In [11]:
collect_driver = DynamicStepDriver(train_env,
                                   agent.collect_policy,
                                   observers = [observer] + training_metrics,
                                   num_steps = 1)

In [18]:
#initial_collect_driver.run()
pre_trained_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


(TimeStep(
 {'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
  'observation': <tf.Tensor: shape=(1, 8), dtype=float32, numpy=
 array([[ 0.23598842, -0.03832475,  0.01568582, -0.00230626, -0.26576924,
         -0.00681765,  1.        ,  1.        ]], dtype=float32)>,
  'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.41610548], dtype=float32)>,
  'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>}),
 ())

# Dataset

In [12]:
dataset = replay_buffer.as_dataset(sample_batch_size = batch_size,
                                   num_steps = 2,
                                   num_parallel_calls = 3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


# Training Loop

In [13]:
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [14]:
policy_dir = os.path.join(os.curdir, "Saved policies")
policy_dir

'.\\Saved policies'

In [15]:
saver = PolicySaver(agent.policy)

In [23]:
agent.train_step_counter.assign(0)
iterator = iter(dataset)

time_step = None
policy_state = agent.policy.get_initial_state(train_env.batch_size)
iterator = iter(dataset)

returns = [-200]

for _ in range(num_iterations):
    time_step, policy_state = collect_driver.run(time_step, policy_state)
    experience, info = next(iterator)
    train_loss = agent.train(experience)
    
    step = agent.train_step_counter.numpy()

    if step % 1000 == 0:
        avg_return = training_metrics[0].result()
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)

    if step % 10000 == 0:
        iteration = step // 1000
        folder = os.path.join(policy_dir, "policy_%dk" % iteration)
        saver.save(folder)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
step = 1000: Average Return = -481.74420166015625
step = 2000: Average Return = -445.8077087402344
step = 3000: Average Return = -382.081787109375
step = 4000: Average Return = -328.1892395019531
step = 5000: Average Return = -227.13723754882812
step = 6000: Average Return = -194.12179565429688
step = 7000: Average Return = -284.3909606933594
step = 8000: Average Return = -274.2027893066406
step = 9000: Average Return = -258.9452209472656


step = 10000: Average Return = -249.7990264892578



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: .\Saved policies\policy_10k\assets


...

step = 141000: Average Return = -51.26951217651367
step = 142000: Average Return = -70.74075317382812
step = 143000: Average Return = -51.87975311279297
step = 144000: Average Return = -84.71998596191406
step = 145000: Average Return = -63.541847229003906
step = 146000: Average Return = -72.99394226074219
step = 147000: Average Return = -87.15099334716797
step = 148000: Average Return = -89.19763946533203
step = 149000: Average Return = -104.53459167480469


step = 150000: Average Return = -94.71683502197266

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: .\Saved policies\policy_150k\assets


INFO:tensorflow:Assets written to: .\Saved policies\policy_150k\assets


# Evaluation and visualization

In [ ]:
plt.plot(returns)
plt.grid()

In [18]:
policy = tf.saved_model.load("Good policies/policy_120k_huber")

In [ ]:
policy = agent.policy

In [22]:
episode_rewards = []
for _ in range(4):
    reward = 0.0
    time_step = eval_env.reset()
    while not time_step.is_last():
        eval_env_py.render()
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step)
        reward += time_step.reward
    episode_rewards.append(reward)

In [23]:
eval_env_py.close()

In [ ]:
tf.reduce_mean(episode_rewards)

In [ ]:
tf.math.reduce_std(episode_rewards)

In [ ]:
tf.reduce_min(episode_rewards)

In [ ]:
tf.reduce_min(returns)

In [ ]:
tf.reduce_mean(returns)

In [ ]:
tf.math.reduce_std(returns)